## Loading the Data

In [58]:
import pandas as pd
import numpy as np
df=pd.read_csv("531all.csv",low_memory=False)
df.head(10)
# df=pd.read_csv("test3.csv",low_memory=False)
# df.head(10)
filtered_df=df[df['Document Number'].fillna('').str.startswith(('IF','IR','IA',"SO"))]
filtered_df=filtered_df[filtered_df['Location']!='Seller Warehouse']
# Single Inventory on hand check
# df_test = filtered_df[filtered_df['Item'].str.contains(r'\d+-\d+-\d+-\d+', regex=True)]
df_test=filtered_df

In [59]:
test1=df[df['Document Number'].str.startswith(("IT",'IA'))]

duplicates = test1[test1.duplicated('Item', keep=False)]
unique_ids=duplicates['Item'].unique()
unique_ids=pd.DataFrame(unique_ids)
unique_ids.columns=['Item has both IT and IA']
unique_ids

,Item has both IT and IA
0,107951-115-10937-12189
1,80000-101-10278-99999
2,108338-101-10843-11559
3,106265-235-10683-99999
4,99608-178-10447-99999
...,...
2677,28580-100-10162-99999
2678,11740-100-10112-99999
2679,13671-100-10104-99999
2680,16150-100-10030-99999


In [60]:
unique_ids.to_excel('Item has both IT and IA.xlsx', index=False, engine='openpyxl')

In [61]:
df[(df['Item']=="28580-100-10162-99999") & (df['Document Number'].str.startswith(("IT",'IA')))]

,Internal ID,Number,Item,Document Number,Date Created,Status,Item Program,Back Ordered,Location,Bin Number,Restock,Quantity,Item Count,On Hand,Available,Expiration Date,Created From
465290,17802,5031805004,28580-100-10162-99999,IA4053,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-New-Jersey,CC-14-A,NaN,800.0,800.0,NaN,NaN,4/30/2021,NaN
465291,17801,5031805004,28580-100-10162-99999,IA4052,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-New-Jersey,CC-15-A,NaN,200.0,200.0,NaN,NaN,4/30/2021,NaN
465292,17800,5031805003,28580-100-10162-99999,IA4051,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-New-Jersey,DD-11-A,NaN,800.0,800.0,NaN,NaN,4/30/2021,NaN
465293,17799,5031805003,28580-100-10162-99999,IA4050,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-New-Jersey,DD-12-A,NaN,125.0,125.0,NaN,NaN,4/30/2021,NaN
465294,17798,5031805005,28580-100-10162-99999,IA4049,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-La-Mirada,E-09-A,NaN,225.0,225.0,NaN,NaN,4/30/2021,NaN
465295,17797,5031805002,28580-100-10162-99999,IA4048,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-La-Mirada,H-05,NaN,1000.0,1000.0,NaN,NaN,3/31/2021,NaN
465296,17796,5031805001,28580-100-10162-99999,IA4047,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-La-Mirada,G-21,NaN,1000.0,1000.0,NaN,NaN,3/31/2021,NaN
465297,17795,5031804009,28580-100-10162-99999,IA4046,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-La-Mirada,A-08-C,NaN,325.0,325.0,NaN,NaN,3/31/2021,NaN
465298,17794,5031804009,28580-100-10162-99999,IA4045,8/31/2018 8:03 pm,NaN,NaN,NaN,GWI-La-Mirada,A-05-D,NaN,75.0,75.0,NaN,NaN,3/31/2021,NaN


## Define functions

In [62]:
# check if status starts with IF
def starts_with_IF(string):
    if string.startswith('IF'):
        return True
    else:
        return False
# check if status starts with IR
def starts_with_IR(string):
    if string.startswith('IR'):
        return True
    else:
        return False
# check if status starts with IT
def starts_with_IT(string):
    if string.startswith('IT'):
        return True
    else:
        return False

# check if status starts with IA
def starts_with_IA(string):
    if string.startswith('IA'):
        return True
    else:
        return False

# check if status starts with CM
def starts_with_CM(string):
    if string.startswith('CM'):
        return True
    else:
        return False
# check if status starts with SO
def starts_with_SO(string):
    if string.startswith('SO'):
        return True
    else:
        return False
    
def set_zero_if_negative(number):
    if number < 0:
        return 0
    else:
        return number
    




In [63]:
# Rule of thumbs FOR ON HAND inventory level

# Location
# Location from seller house and is not inventory receipt should not count

# IR Not to count (based on bin number)
# restock is no then not count

# CM
# No need to count CM

# IF, status not equal to shiped, on hand level would not be changed

# IA has no condition that does not count to inventory: 
# return to seller example 30018-100-10343-11248 and on hold example 82801-240-10131-11334
# Investigate---example: 62300-100-10020-10984
# Damaged needed to be removed:67010-101-10272-99999

## On hand Inventory check

In [64]:
# Calculate the Inventory level (on hand) at each transaction

def on_hand(df_test):
    running_sum=0
    for i in range(1,len(df_test)+1):
        if(df_test.loc[df_test.index[-i],'Location'] in ["Seller Warehouse"]):
            if(starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
                if (df_test.loc[df_test.index[-i],'Restock'] == 'Yes'):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
        elif (
            starts_with_IF(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Inventory Fullfillment"
            if(
                df_test.loc[df_test.index[-i],'Status'] in ["Shipped"]): # status is  shipped
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
        elif(
            starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
            if(df_test.loc[df_test.index[-i],'Restock'] == 'No'):
                running_sum=running_sum #The on hand lvl remains the same
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
        elif (
                starts_with_SO(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Sales order"
                running_sum=running_sum
        else:
            running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
    running_sum=set_zero_if_negative(running_sum) # ignore negative counts
    return running_sum

result=df_test.groupby('Item').apply(on_hand)
result=pd.DataFrame(result)
new_column_names = ['OnhandCount']
result.columns = new_column_names
result

,OnhandCount
Item,
05-5008R,0.0
05-5010R,500.0
05-537R,35200.0
05-538R,0.0
05-540R,0.0
...,...
YS-ltpC24010312,0.0
YS-ltpC24010313,900.0
YS-mbp0827,0.0


In [65]:
# Extracting the constant AltTotal for each ID
NetOnHand= df[['Item', 'On Hand']].drop_duplicates()
NetOnHand['On Hand'].fillna(0,inplace=True)

# Merging the results with the constant AltTotal values
final_result = pd.merge(result, NetOnHand, on='Item')
final_result['OnhandCount']=final_result['OnhandCount'].round(3)
final_result['On Hand']=final_result['On Hand'].round(3)
final_result

,Item,OnhandCount,On Hand
0,05-5008R,0.0,0.0
1,05-5010R,500.0,500.0
2,05-537R,35200.0,35200.0
3,05-538R,0.0,0.0
4,05-540R,0.0,0.0
...,...,...,...
10455,YS-ltpC24010312,0.0,0.0
10456,YS-ltpC24010313,900.0,900.0
10457,YS-mbp0827,0.0,0.0
10458,YS-spp041106,0.0,0.0


In [66]:
# Compare Column1 and Column2, store the result in a new column
final_result['OComparison'] = final_result['OnhandCount'] == final_result['On Hand']
final_result
# Count the number of True and False values
count_result = final_result['OComparison'].value_counts()
print(count_result)

accuracy = count_result[True] / final_result.shape[0]
print("Accuracy:",accuracy)


OComparison
True     10452
False        8
Name: count, dtype: int64
Accuracy: 0.9992351816443594


In [67]:
final_result

,Item,OnhandCount,On Hand,OComparison
0,05-5008R,0.0,0.0,True
1,05-5010R,500.0,500.0,True
2,05-537R,35200.0,35200.0,True
3,05-538R,0.0,0.0,True
4,05-540R,0.0,0.0,True
...,...,...,...,...
10455,YS-ltpC24010312,0.0,0.0,True
10456,YS-ltpC24010313,900.0,900.0,True
10457,YS-mbp0827,0.0,0.0,True
10458,YS-spp041106,0.0,0.0,True


In [68]:
(final_result['OnhandCount']<0).value_counts()

OnhandCount
False    10460
Name: count, dtype: int64

In [69]:
final_result[final_result['OnhandCount']<0]

,Item,OnhandCount,On Hand,OComparison


In [70]:
final_result[final_result['Item']=='109764-101-10334-99999']

,Item,OnhandCount,On Hand,OComparison
2956,109764-101-10334-99999,4850.0,4850.0,True


In [71]:
false_onhand=final_result[final_result['OComparison']==False]
unique_id_false_onhand=set(false_onhand['Item'])

In [72]:
false_onhand

,Item,OnhandCount,On Hand,OComparison
1090,107166-109-10680-11732,88.0,89.0,False
3334,110320-124-10937-12189,0.2,0.7,False
4679,18911-100-10343-11248,0.0,425.0,False
6516,43900-103-10012-11426,2030.0,2280.0,False
7627,69584-133-10036-11456,90.0,0.0,False
7667,69822-100-10108-99999,0.0,-25.0,False
9931,99424-350-10770-99999,50.0,0.0,False
10103,99889-100-10328-99999,1500.0,2000.0,False


In [73]:
Filter_mismatch = df_test[df_test['Item'].isin(unique_id_false_onhand)]
id_counts = Filter_mismatch['Item'].value_counts()
print(id_counts)

Item
43900-103-10012-11426     379
107166-109-10680-11732    197
18911-100-10343-11248     155
69822-100-10108-99999      94
99889-100-10328-99999      30
110320-124-10937-12189     10
69584-133-10036-11456       9
99424-350-10770-99999       1
Name: count, dtype: int64


In [74]:
# Error analysis
# 69822-100-10108-99999 has negative on hand in system
# 99424-350-10770-99999 IF has positive item count
# 99889-100-10328-99999	

## Availability calculation with Sales order priority 99.3% accuracy

In [75]:
# Availability calculation current logic 99.3% accuracy True  10545  False   69

# Filtered data with "SO",'IF','IR','IA',"IT"
# Location
# Location from seller Warehouse only count with IR with Restock that is Yes

# SO(Sales Order status)
# Canceld/ Pending Approval does not count
# Closed need to record and consider it in IF
# ohter status need to be counted

# IF 

# 1. Count when the created from a SO that is not recorded in the data (So we need to create a list for all recored SO entries. i.e SO_Check) 
# 2. Count when it is created from a closed SO and is shipped
# 3. Count when it is not created from SO but other forms
# 4. Count the exceeding number when the fulfillment exceeding the corresponding SO (Record and update item number from each SO and count IF if it exceed)
# (*be aware multiple IF might be created from same SO and invoice usually match the IF since it comes after IF)

#IR 
#when bin number is None item should not count to the available.  (But some of them has no bin number still counted)
#When Location ENDS WITH suffix=["QC", "QU","FDA Hold","Hold","Returns"], should not counted to the available （all corresponding condition were checed)


#IA
#When Location ENDS WITH suffix=["QC", "QU","FDA Hold","Hold","Returns"], should not counted to the available

# IT
# lOCATION THAT ENDS WITH suffix=["QC", "QU","FDA Hold","Hold","Returns"] need to be counted negatively to the available

# Eveything else in the filtered data needs to be counted.


In [76]:
filtered_df=df[df['Document Number'].fillna('').str.startswith(('IF','IR','IA',"SO","IT"))]
# filtered_df=filtered_df[filtered_df['Location']!='Seller Warehouse']
# Single Inventory on hand check
df_test1=filtered_df

In [77]:
def ends_with_any(string, suffix_list):
    # Check if the string ends with any of the strings in suffix_list
    for suffix in suffix_list:
        if string.endswith(suffix):
            return True
    return False
suffix=["QC", "QU","FDA Hold","Hold","Returns"]

def starts_with_Sales(string):
    if string.startswith('Sales'):
        return True
    else:
        return False


In [78]:
def add_value(array, id, value_to_add):
    for i, (current_id, current_value) in enumerate(array):
        if current_id == id:
            array[i] = (current_id, current_value + value_to_add)
            break
    else:  # This else belongs to the for loop, not the if statement
        array.append((id, value_to_add))

# #Example:
# array=[]
# data = {
#     'Entry': ['SO-001', 'SO-002', 'IF-100', 'IF-101', 'IF-102', 'IF-103'],
#     'Item Count': [10, 15, 6, 4, 15, 2],
#     'Created From': ['', '', 'SO-001', 'SO-001', 'SO-002', 'SO-002']
# }
# result=add_value(array,'SO-002',-150)
# result=add_value(array,'SO-001',-300)
# array

In [79]:
def update_SO_Value(array, id, new_value):
    for i, (current_id, current_value) in enumerate(array):
        if current_id == id:
            if current_value < 0:
                updated_value = current_value+new_value
                array[i] = (current_id, updated_value)
                return updated_value if updated_value > 0 else 0
            else:
                updated_value = current_value + new_value
                array[i] = (current_id, updated_value)
                return new_value
            break  # Exit if ID is found

#     If the ID is not found, do nothing and implicitly return None

# # Example usage:
# # array = [(1, -100), (2, -200), (3, -300)]
# 
# result = update_SO_Value(array, 'SO-002', 140)  
# print(f"Result: {result}")
# print(array)
# result = update_SO_Value(array,'SO-002', 250)  
# print(f"Result: {result}")
# print(array)
# result = update_SO_Value(array,'SO-001', 300) 
# print(f"Result: {result}")
# print(array)
# result = update_SO_Value(array,'SO-001', 150)  
# print(f"Result: {result}")
# print(array)
# result = update_SO_Value(array, 4, 150)  
# print(f"Result: {result}")
# print(array)
# 
# print('SO-001' in array)

In [80]:
# Calculate the Inventory available at each transaction
def ava_check(df_test):
    Closed_Cases=[]
    SO_check=[]
    miss_so_detail=[]
    excess_ship=[]
    running_sum=0
    for i in range(1,len(df_test)+1):
        # If location is in Seller Warehouse
        if(df_test.loc[df_test.index[-i],'Location'] in ["Seller Warehouse"]):
            if(starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
                if (df_test.loc[df_test.index[-i],'Restock'] == 'Yes'):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    running_sum=running_sum        
            else:
                running_sum=running_sum
        # If transaction type is "Sales order"
        elif (
                starts_with_SO(df_test.loc[df_test.index[-i],'Document Number'])):
            # Add to available count array
            add_value(SO_check,df_test.loc[df_test.index[-i],'Document Number'],df_test.loc[df_test.index[-i],'Item Count'])
            if(df_test.loc[df_test.index[-i],'Status'] in 
               ["Cancelled","Pending Approval"]):
                running_sum=running_sum
            elif(df_test.loc[df_test.index[-i],'Status'] in 
               ["Pending Fulfillment"]):
                running_sum+=df_test.loc[df_test.index[-i],'Item Count']
            elif(df_test.loc[df_test.index[-i],'Status'] == "Closed"):
                Closed_Cases.append(df_test.loc[df_test.index[-i],'Document Number'])
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
        # If transaction type is "Inventory Fullfillment"
        elif (
            starts_with_IF(df_test.loc[df_test.index[-i],'Document Number'])): 
            if(starts_with_Sales(df_test.loc[df_test.index[-i],'Created From'])):
                part_after_sales_order = (df_test.loc[df_test.index[-i],'Created From']).split("Sales Order #")[1].strip()
                if(part_after_sales_order not in (id for id, _ in SO_check)):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                else:
                    if(part_after_sales_order in Closed_Cases and df_test.loc[df_test.index[-i],'Status']== "Shipped"):
                        running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    else:
                        result=update_SO_Value(SO_check,part_after_sales_order,df_test.loc[df_test.index[-i],'Quantity'])
                        if (result >0):
                            running_sum-=result
                            miss_so_detail.append(part_after_sales_order)
                            excess_ship.append(df_test.loc[df_test.index[-i],'Document Number'])
                            
            # If it is not created from sales order
            else: 
                running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level that created from not start with sales
        # If transaction type is "Inventory Receipt"
        elif(
            starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  
            if( 
                # df_test.loc[df_test.index[-i],'Bin Number'] is np.nan  #If Bin number is null
                df_test.loc[df_test.index[-i],'Restock'] == 'No'
                ):
                    running_sum=running_sum #The on hand lvl remains the same
            elif(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                    running_sum=running_sum #The on hand lvl remains the same
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
        # If transaction type is "Inventory Transfer"
        elif(
            starts_with_IT(df_test.loc[df_test.index[-i],'Document Number'])):  
            if(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                running_sum-=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
        # If transaction type is "Inventory Ajustment"
        elif(
            starts_with_IA(df_test.loc[df_test.index[-i],'Document Number'])):  
            if(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                running_sum=running_sum
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
        else:
            running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
    # running_sum=set_zero_if_negative(running_sum) # ignore negative counts
    return running_sum,miss_so_detail,excess_ship






In [81]:
# Apply function to calculate the available level, situation of missing detail, shipment that is excess than the SO(inventory detail) record
A_result=df_test1.groupby('Item').apply(ava_check)
A_result=pd.DataFrame(A_result)
new_column_names = ['qty check']
A_result.columns = new_column_names
A_result=A_result['qty check'].apply(pd.Series)

### Check situations that SO lack of Inventory detail but IF has completed transaction

In [86]:
A_result.columns=['ava_check','SO_lack_InvDetail','Related IF']

In [87]:
A_result[A_result['SO_lack_InvDetail'].apply(len) > 0]

,ava_check,SO_lack_InvDetail,Related IF
Item,,,
10081-100-10043-10111,0.000000e+00,"[SO-GWI-8558, SO-GWI-91174]","[IF7394, IF86071]"
105852-100-10333-11520,0.000000e+00,[SO-GWI-102984],[IF96986]
106007-100-10104-10605,0.000000e+00,[SO-GWI-105026],[IF100155]
106116-142-10653-12230,1.734723e-18,[SO-GWI-97960],[IF92362]
106295-101-10688-10241,0.000000e+00,[SO-GWI-59562],[IF55849]
...,...,...,...
97131-101-10025-99999,0.000000e+00,[SO-GWI-94484],[IF89778]
97142-101-10562-99999,0.000000e+00,[SO-GWI-96806],[IF91217]
97220-160-10814-99999,1.387779e-17,[SO-GWI-104050],[IF97962]


In [88]:
# Extracting the constant AltTotal for each ID
NetOnHand= df[['Item', 'Available']].drop_duplicates()
NetOnHand['Available'].fillna(0,inplace=True)

# Merging the results with the constant AltTotal values
final_A_result = pd.merge(A_result['ava_check'], NetOnHand, on='Item')
final_A_result['ava_check']=final_A_result['ava_check'].round(3)
final_A_result['Available']=final_A_result['Available'].round(3)


In [89]:
final_A_result

,Item,ava_check,Available
0,05-5008R,0.0,0.0
1,05-5010R,500.0,500.0
2,05-537R,27200.0,27200.0
3,05-538R,0.0,0.0
4,05-540R,0.0,0.0
...,...,...,...
11028,YS-ltpC24010312,0.0,0.0
11029,YS-ltpC24010313,900.0,900.0
11030,YS-mbp0827,0.0,0.0
11031,YS-spp041106,0.0,0.0


In [90]:
# Filter out item starst with 'SKU', bc netsuite did not provide any availability on them
for index, row in final_A_result.iterrows():
    if row['Item'].startswith('SKU'):
        final_A_result.at[index, 'ava_check'] = 0


In [91]:
# Compare Column1 and Column2, store the result in a new column
final_A_result['AComparison'] = final_A_result['ava_check'] == final_A_result['Available']
# Count the number of True and False values
count_A_result = final_A_result['AComparison'].value_counts()
print(count_A_result)

accuracy = count_A_result[True] / final_A_result.shape[0]
print("Accuracy:",accuracy)

AComparison
True     11000
False       33
Name: count, dtype: int64
Accuracy: 0.9970089730807578


In [92]:
df_test1[df_test1['Document Number']=='SO-GWI-8558']

,Internal ID,Number,Item,Document Number,Date Created,Status,Item Program,Back Ordered,Location,Bin Number,Restock,Quantity,Item Count,On Hand,Available,Expiration Date,Created From
435500,107713,C3-0750-180823,10081-100-10043-10111,SO-GWI-8558,2/18/2019 8:15 am,Billed,NaN,NaN,GWI-New-Jersey,NaN,NaN,175.0,-175.0,NaN,NaN,8/22/2020,NaN
435501,107711,C561705032,52920-100-10173-10062,SO-GWI-8558,2/18/2019 8:15 am,Billed,NaN,NaN,GWI-New-Jersey,NaN,NaN,50.0,-50.0,NaN,NaN,5/14/2019,NaN
435502,107712,TMF18003,24270-101-10255-11148,SO-GWI-8558,2/18/2019 8:15 am,Billed,NaN,NaN,GWI-New-Jersey,NaN,NaN,825.0,-825.0,NaN,NaN,8/30/2020,NaN
435503,107712,TMF18004,24270-101-10255-11148,SO-GWI-8558,2/18/2019 8:15 am,Billed,NaN,NaN,GWI-New-Jersey,NaN,NaN,1175.0,-1175.0,NaN,NaN,11/2/2020,NaN
435504,107713,C3-0750-181017,10081-100-10043-10111,SO-GWI-8558,2/18/2019 8:15 am,Billed,NaN,NaN,GWI-New-Jersey,NaN,NaN,15.0,-15.0,NaN,NaN,10/16/2020,NaN


In [93]:
df_test1[df_test1['Document Number']=='IF7394']

,Internal ID,Number,Item,Document Number,Date Created,Status,Item Program,Back Ordered,Location,Bin Number,Restock,Quantity,Item Count,On Hand,Available,Expiration Date,Created From
435319,108105,TMF18004,24270-101-10255-11148,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-A-09-A,NaN,175.0,-175.0,NaN,NaN,11/2/2020,Sales Order #SO-GWI-8558
435320,108106,C3-0750-181017,10081-100-10043-10111,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-C-33-A,NaN,125.0,-125.0,NaN,NaN,10/16/2020,Sales Order #SO-GWI-8558
435321,108106,C3-0750-180823,10081-100-10043-10111,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-A-08-A,NaN,75.0,-75.0,NaN,NaN,8/22/2020,Sales Order #SO-GWI-8558
435322,108104,C561705032,52920-100-10173-10062,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-F-06-B,NaN,50.0,-50.0,NaN,NaN,5/14/2019,Sales Order #SO-GWI-8558
435323,108105,TMF18003,24270-101-10255-11148,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-B-22-B,NaN,825.0,-825.0,NaN,NaN,8/30/2020,Sales Order #SO-GWI-8558
435324,108105,TMF18004,24270-101-10255-11148,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-A-09-C,NaN,1000.0,-1000.0,NaN,NaN,11/2/2020,Sales Order #SO-GWI-8558
435325,108105,TMF18004,24270-101-10255-11148,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-A-08-B,NaN,500.0,-500.0,NaN,NaN,11/2/2020,Sales Order #SO-GWI-8558
435326,108106,C3-0750-180823,10081-100-10043-10111,IF7394,2/19/2019 5:27 am,Shipped,NaN,NaN,GWI-New-Jersey,N-A-09-A,NaN,100.0,-100.0,NaN,NaN,8/22/2020,Sales Order #SO-GWI-8558


In [94]:
Mismatch_inventory=final_A_result[(final_A_result['AComparison']==False)]
Mismatch_inventory

,Item,ava_check,Available,AComparison
13,10002-101-10485-10763,113445.00,113446.0,False
1154,107166-109-10680-11732,86.00,87.0,False
2089,108288-111-10708-10260,1840.00,1800.0,False
2740,109089-100-11078-12809,175.00,0.0,False
3556,110150-102-10551-99999,650.00,625.0,False
3658,110320-124-10937-12189,0.10,0.6,False
3806,11600-100-10293-99999,750.00,6350.0,False
4264,13450-100-10249-99999,-50.00,0.0,False
4336,14050-102-10491-99999,300.00,0.0,False
4688,16610-104-10044-99999,120.00,140.0,False


In [95]:
Mis_id=Mismatch_inventory.iloc[:,0].tolist()

In [96]:
Mis_id=Mis_id[:-1]

In [97]:
Filter_mismatch=df_test1[df_test1['Item'].isin(Mis_id)]
id_counts=Filter_mismatch['Item'].value_counts()
print(id_counts)

Item
70130-104-10054-99999     1302
69430-102-10408-99999      833
36740-139-10327-99999      614
43900-103-10012-11426      391
17070-104-10046-99999      294
14050-102-10491-99999      277
11600-100-10293-99999      272
108288-111-10708-10260     222
107166-109-10680-11732     197
18911-100-10343-11248      163
16610-104-10044-99999      161
69584-133-10036-11456      111
80001-100-10606-10062       68
109089-100-11078-12809      64
55111-101-10752-99999       56
10002-101-10485-10763       55
70330-101-10001-11575       47
13450-100-10249-99999       44
48360-100-10002-99999       31
91501-205-10417-11626       23
97421-201-10452-11671       22
77617-201-10726-99999       20
110150-102-10551-99999      17
25851-142-10435-36371       15
97802-201-10124-11490       14
77615-201-10726-99999       11
110320-124-10937-12189      10
25882-201-10554-11799        6
37800-102-10244-77777        5
21459-178-10452-11671        4
87905-200-10417-99999        1
99424-350-10770-99999        1
Nam

In [98]:
final_A_result[(final_A_result['AComparison']==False) & (final_A_result['ava_check']<0)]                                                                                            

,Item,ava_check,Available,AComparison
4264,13450-100-10249-99999,-50.00,0.0,False
5034,18911-100-10343-11248,-175.00,425.0,False
5234,21459-178-10452-11671,-200.00,0.0,False
5535,25851-142-10435-36371,-0.05,0.0,False
5541,25882-201-10554-11799,-100.00,0.0,False
6308,37800-102-10244-77777,-17500.00,0.0,False
7200,48360-100-10002-99999,-25.00,0.0,False
8235,70130-104-10054-99999,-20.00,0.0,False
8414,70330-101-10001-11575,-2050.00,0.0,False
8534,77615-201-10726-99999,-0.40,0.0,False


In [99]:
final_A_result[final_A_result['Item']=='106883-104-10496-11712']

,Item,ava_check,Available,AComparison
869,106883-104-10496-11712,0.0,0.0,True


In [100]:
final_ALL_result = pd.merge(final_result, final_A_result, on='Item')

In [101]:
final_ALL_result[(final_ALL_result['On Hand']<final_ALL_result['ava_check'])]

,Item,OnhandCount,On Hand,OComparison,ava_check,Available,AComparison
7627,69584-133-10036-11456,90.0,0.0,False,90.0,0.0,False
9931,99424-350-10770-99999,50.0,0.0,False,50.0,0.0,False


In [102]:
final_ALL_result[final_ALL_result['Item']=='99384-104-10718-99999']

,Item,OnhandCount,On Hand,OComparison,ava_check,Available,AComparison
9903,99384-104-10718-99999,0.0,0.0,True,0.0,0.0,True


In [103]:
final_ALL_result[final_ALL_result['AComparison']==False]

,Item,OnhandCount,On Hand,OComparison,ava_check,Available,AComparison
13,10002-101-10485-10763,113471.0,113471.0,True,113445.00,113446.0,False
1090,107166-109-10680-11732,88.0,89.0,False,86.00,87.0,False
1951,108288-111-10708-10260,1840.0,1840.0,True,1840.00,1800.0,False
2476,109089-100-11078-12809,325.0,325.0,True,175.00,0.0,False
3236,110150-102-10551-99999,650.0,650.0,True,650.00,625.0,False
3334,110320-124-10937-12189,0.2,0.7,False,0.10,0.6,False
3481,11600-100-10293-99999,6350.0,6350.0,True,750.00,6350.0,False
3929,13450-100-10249-99999,0.0,0.0,True,-50.00,0.0,False
3999,14050-102-10491-99999,300.0,300.0,True,300.00,0.0,False
4338,16610-104-10044-99999,180.0,180.0,True,120.00,140.0,False


In [104]:

# Available checking list
# Test for IT
# 99551-101-10025-99999
# 13670-100-10104-99999


# SO closed but IF packed has to shipped then count only one special case ex.CPE-095 G
# IF bin number OnHold AND OOS/Failed has problem 107688-100-10343-11248
# CM does not need to included in available count ex. 108107-102-10708-10260
# Sales order= 0.03 but shipped 0.05 106834-160-10496-11712
# 99553-105-10897-99999
# 11593-101-10300-10062
# 13670-100-10104-99999
# 105852-100-10333-11520

#IA should not count suffix examples
# 106134-100-10471-11705
# 106158-104-10496-11712
# 106702-100-10496-11712

# 99868-105-11101-99999 bin number damaged

# 99384-104-10718-99999 SO-GWI-109343 IF103043 fullfillment is greater than the SO
# 48360-100-10002-99999 item fullfillment is less than SO IF33786	SO-GWI-36105

#10002-102-10328-99999 IF but no corresponding SO

#Netsuite calculation error
#106029-101-10236-11286
#107066-201-10731-12306
#108165-217-10955-12595
#109692-100-10937-12189 SO PENDING FULLFILLMENT SHOULD BE UNAVAILABLE
#109764-101-10334-99999 when system haven't documented netsuite already calculated
# 99697-124-11070-99999  missing bin number 

# IT Item count mess 
# IR Testing not to count because IT double counted 106324-100-10333-11520 IR62043 & IT1284

# Missing 5 inventory were not in the record 
#109334-100-10245-12874


#Contains both IT and IA
# 1000001-101-10740-99999
# 10002-101-10485-10763
# 40300-100-10343-11248
# 85800-142-10367-99999
# 41930-101-10509-11736

## Single Item  check (updated)

In [105]:
# Calculate the Inventory level (on hand) at each transaction

def single_on_hand(df_test):
    running_sum=0
    for i in range(1,len(df_test)+1):
        if(df_test.loc[df_test.index[-i],'Location'] in ["Seller Warehouse"]):
            if(starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
                if (df_test.loc[df_test.index[-i],'Restock'] == 'Yes'):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    onhand_lvl.append(running_sum)
                else:
                    onhand_lvl.append(running_sum)
            else:
                onhand_lvl.append(running_sum)
        elif (
            starts_with_IF(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Inventory Fullfillment"
            if(
                df_test.loc[df_test.index[-i],'Status'] in ["Shipped"]): # status is  shipped
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                onhand_lvl.append(running_sum)
            else:
                onhand_lvl.append(running_sum)

        elif(
            starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
            if(df_test.loc[df_test.index[-i],'Restock'] == 'No'):
                onhand_lvl.append(running_sum)
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
                onhand_lvl.append(running_sum)
             # If transaction type is "Inventory Transfer"
        elif(
            starts_with_IT(df_test.loc[df_test.index[-i],'Document Number'])):
            running_sum-=df_test.loc[df_test.index[-i],'Item Count'] # Here is only for one by one checking no need to count in real
            onhand_lvl.append(running_sum)
        elif (
                starts_with_SO(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Sales order"
                onhand_lvl.append(running_sum)
        else:
            running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
            onhand_lvl.append(running_sum)
#     running_sum=set_zero_if_negative(running_sum) # ignore negative counts
    return onhand_lvl



In [106]:

# Calculate the Inventory available at each transaction
def single_ava_check(df_test):
    Closed_Cases=[]
    SO_check=[]
    running_sum=0
    for i in range(1,len(df_test)+1):
        # If location is in Seller Warehouse
        if(df_test.loc[df_test.index[-i],'Location'] in ["Seller Warehouse"]):
            if(starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
                if (df_test.loc[df_test.index[-i],'Restock'] == 'Yes'):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    Inventory_lvl.append(running_sum)
                else:
                    Inventory_lvl.append(running_sum)
            else:
                Inventory_lvl.append(running_sum)
        # If transaction type is "Sales order"
        elif (
                starts_with_SO(df_test.loc[df_test.index[-i],'Document Number'])):
            # Add to available count array
            add_value(SO_check,df_test.loc[df_test.index[-i],'Document Number'],df_test.loc[df_test.index[-i],'Item Count'])
            if(df_test.loc[df_test.index[-i],'Status'] in 
               ["Cancelled","Pending Approval"]):
                Inventory_lvl.append(running_sum)
            elif(df_test.loc[df_test.index[-i],'Status'] in 
               ["Pending Fulfillment"]):
                running_sum+=df_test.loc[df_test.index[-i],'Item Count']
                Inventory_lvl.append(running_sum)
            elif(df_test.loc[df_test.index[-i],'Status'] == "Closed"):
                Closed_Cases.append(df_test.loc[df_test.index[-i],'Document Number'])
                Inventory_lvl.append(running_sum)
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
        # If transaction type is "Inventory Fullfillment"
        elif (
            starts_with_IF(df_test.loc[df_test.index[-i],'Document Number'])): 
            if(starts_with_Sales(df_test.loc[df_test.index[-i],'Created From'])):
                part_after_sales_order = (df_test.loc[df_test.index[-i],'Created From']).split("Sales Order #")[1].strip()
                if(part_after_sales_order not in (id for id, _ in SO_check)):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    Inventory_lvl.append(running_sum)
                else:
                    if(part_after_sales_order in Closed_Cases and df_test.loc[df_test.index[-i],'Status']== "Shipped"):
                        running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                        Inventory_lvl.append(running_sum)
                    else:
                        result=update_SO_Value(SO_check,part_after_sales_order,df_test.loc[df_test.index[-i],'Quantity'])
                        if (result >0):
                            running_sum-=result
                            Inventory_lvl.append(running_sum)
                        else:
                            Inventory_lvl.append(running_sum)
            # If it is not created from sales order
            else: 
                running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level that created from not start with sales
                Inventory_lvl.append(running_sum)
        # If transaction type is "Inventory Receipt"
        elif(
             starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  
            if( 
                df_test.loc[df_test.index[-i],'Bin Number'] is np.nan  #If Bin number is null
                ):
                    running_sum=running_sum #The on hand lvl remains the same
                    Inventory_lvl.append(running_sum)
            elif(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                    running_sum=running_sum #The on hand lvl remains the same
                    Inventory_lvl.append(running_sum)
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
        # If transaction type is "Inventory Transfer"
        elif(
            starts_with_IT(df_test.loc[df_test.index[-i],'Document Number'])):  
            if(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                running_sum-=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
            else:
                Inventory_lvl.append(running_sum)
                
        # If transaction type is "Inventory Ajustment"
        elif(
            starts_with_IA(df_test.loc[df_test.index[-i],'Document Number'])):  
            if(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                running_sum=running_sum
                Inventory_lvl.append(running_sum)
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
        else:
            running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
            Inventory_lvl.append(running_sum)
#     running_sum=set_zero_if_negative(running_sum) # ignore negative counts
    return Inventory_lvl



In [107]:
onhand_lvl=[]
Inventory_lvl=[]
filtered_df=df[df['Document Number'].fillna('').str.startswith(('IF','IR','IA',"SO","IT"))]
# Single Inventory on hand check
df_test1=filtered_df

In [108]:
Itemcheck='37620-115-10460-99999'
single_on_hand(df_test1[df_test1['Item']==Itemcheck])
df_test_single=df_test1[df_test1['Item']==Itemcheck]

In [109]:
df_test_single.insert(11,'Onhand_lvl',onhand_lvl[::-1]) # Insert the reverse of the list

In [110]:
single_ava_check(df_test1[df_test1['Item']==Itemcheck])
df_test_single.insert(13,'Available_lvl',Inventory_lvl[::-1]) # Insert the reverse of the list
df_test_single

,Internal ID,Number,Item,Document Number,Date Created,Status,Item Program,Back Ordered,Location,Bin Number,Restock,Onhand_lvl,Quantity,Available_lvl,Item Count,On Hand,Available,Expiration Date,Created From
1837,9886101,7122-2402001,37620-115-10460-99999,IF127781,5/24/2024 12:59 pm,Shipped,Non SD Program,NaN,GWI-Chino,RG-38-A,NaN,95.0,5.0,95.0,-5.0,95.0,95.0,2/21/2026,Sales Order #SO-GWI-134781
1887,9884467,7122-2402001,37620-115-10460-99999,SO-GWI-134781,5/24/2024 11:56 am,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,100.0,5.0,95.0,-5.0,95.0,95.0,2/21/2026,NaN
8453,9758489,7122-2402001,37620-115-10460-99999,IF126335,5/9/2024 12:24 pm,Shipped,Non SD Program,NaN,GWI-Chino,RG-38-A,NaN,100.0,5.0,100.0,-5.0,95.0,95.0,2/21/2026,Sales Order #SO-GWI-133145
8795,9751995,7122-2402001,37620-115-10460-99999,SO-GWI-133145,5/8/2024 1:49 pm,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,105.0,5.0,100.0,-5.0,95.0,95.0,2/21/2026,NaN
10939,9697005,7122-2402001,37620-115-10460-99999,IF125634,5/1/2024 5:59 pm,Shipped,Non SD Program,NaN,GWI-Chino,RG-38-A,NaN,105.0,5.0,105.0,-5.0,95.0,95.0,2/21/2026,Sales Order #SO-GWI-132497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404169,398690,7122-1906002,37620-115-10460-99999,IF19217,11/7/2019 9:19 am,Shipped,NaN,NaN,GWI-La-Mirada,Z-02,NaN,185.0,10.0,185.0,-10.0,95.0,95.0,6/17/2021,Sales Order #SO-GWI-20652
404457,396445,7122-1906002,37620-115-10460-99999,SO-GWI-20652,11/5/2019 4:30 pm,Billed,NaN,NaN,GWI-La-Mirada,NaN,NaN,195.0,10.0,185.0,-10.0,95.0,95.0,6/17/2021,NaN
405031,391399,7122-1906002,37620-115-10460-99999,IF18913,11/1/2019 8:27 am,Shipped,NaN,NaN,GWI-La-Mirada,Z-02,NaN,195.0,5.0,195.0,-5.0,95.0,95.0,6/17/2021,Sales Order #SO-GWI-20422
405105,390534,7122-1906002,37620-115-10460-99999,SO-GWI-20422,10/31/2019 4:10 pm,Billed,NaN,NaN,GWI-La-Mirada,NaN,NaN,200.0,5.0,195.0,-5.0,95.0,95.0,6/17/2021,NaN


In [111]:
df_test_single[df_test_single['Quantity']==25]

,Internal ID,Number,Item,Document Number,Date Created,Status,Item Program,Back Ordered,Location,Bin Number,Restock,Onhand_lvl,Quantity,Available_lvl,Item Count,On Hand,Available,Expiration Date,Created From
28802,9327680,7122-2402001,37620-115-10460-99999,IF121156,3/12/2024 5:43 pm,Shipped,Non SD Program,NaN,GWI-Chino,RG-38-A,NaN,175.0,25.0,175.0,-25.0,95.0,95.0,2/21/2026,Sales Order #SO-GWI-127840
28852,9326618,7122-2402001,37620-115-10460-99999,SO-GWI-127840,3/12/2024 3:07 pm,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,200.0,25.0,175.0,-25.0,95.0,95.0,2/21/2026,NaN
46904,8907116,7122-2303001,37620-115-10460-99999,IF116623,1/19/2024 2:59 pm,Shipped,Non SD Program,NaN,GWI-Chino,RA-10-C,NaN,0.0,25.0,-180.0,-25.0,95.0,95.0,3/1/2025,Sales Order #SO-GWI-122465
49213,8851474,7122-2303001,37620-115-10460-99999,SO-GWI-122465,1/15/2024 9:01 am,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,95.0,25.0,70.0,-25.0,95.0,95.0,3/1/2025,Quote #CQ-GWI-21045
133539,6830814,7122-2211001,37620-115-10460-99999,IF94637,4/21/2023 11:50 am,Shipped,Non SD Program,NaN,GWI-Chino,Shipping Staging,NaN,100.0,25.0,100.0,-25.0,95.0,95.0,11/6/2024,Sales Order #SO-GWI-100051
134978,6793332,7122-2211001,37620-115-10460-99999,SO-GWI-100051,4/18/2023 8:17 am,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,125.0,25.0,100.0,-25.0,95.0,95.0,11/6/2024,Quote #CQ-GWI-16482
160931,6032888,7122-2110001A,37620-115-10460-99999,IF87102,1/13/2023 2:26 pm,Shipped,Non SD Program,NaN,GWI-Chino,Shipping Staging,NaN,35.0,25.0,35.0,-25.0,95.0,95.0,10/6/2023,Sales Order #SO-GWI-92027
161985,5997295,7122-2110001A,37620-115-10460-99999,SO-GWI-92027,1/10/2023 1:39 pm,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,60.0,25.0,35.0,-25.0,95.0,95.0,10/6/2023,Quote #CQ-GWI-14869
219851,4439183,7122-2101001,37620-115-10460-99999,IF70721,4/25/2022 8:29 pm,Shipped,Non SD Program,NaN,GWI-Chino,RJ-21-A,NaN,50.0,25.0,50.0,-25.0,95.0,95.0,1/27/2023,Sales Order #SO-GWI-75143
220434,4421407,7122-2101001,37620-115-10460-99999,SO-GWI-75143,4/22/2022 12:56 pm,Billed,Non SD Program,NaN,GWI-Chino,NaN,NaN,75.0,25.0,50.0,-25.0,95.0,95.0,1/27/2023,Quote #CQ-GWI-12014


In [112]:
# Display all dataset
pd.set_option('display.max_rows', None)

In [113]:
import pandas as pd

# Sample data
data = {
    'Entry': ['SO-001', 'SO-002', 'IF-100', 'IF-101', 'IF-102', 'IF-103'],
    'Item Count': [10, 15, 6, 4, 15, 2],
    'Created From': ['', '', 'SO-001', 'SO-001', 'SO-002', 'SO-002']
}

# Load data into a DataFrame
df = pd.DataFrame(data)

# Initialize tracking variables
so_counts = {}
available_level = 0
available_levels = []

# Iterate through each row
for index, row in df.iterrows():
    entry, item_count, created_from = row['Entry'], row['Item Count'], row['Created From']

    if entry.startswith('SO'):
        # Update SO count and add to available level
        so_counts[entry] = item_count
        available_level += item_count

    elif entry.startswith('IF'):
        # Check and update available level based on IF count
        so_count = so_counts.get(created_from, 0)
        if item_count > so_count:
            available_level += item_count - so_count
        so_counts[created_from] = max(0, so_count - item_count)

    # Store the current available level
    available_levels.append(available_level)

# Add cumulative available level to DataFrame
df['Available Level After Entry'] = available_levels

print(df)


    Entry  Item Count Created From  Available Level After Entry
0  SO-001          10                                        10
1  SO-002          15                                        25
2  IF-100           6       SO-001                           25
3  IF-101           4       SO-001                           25
4  IF-102          15       SO-002                           25
5  IF-103           2       SO-002                           27


In [114]:
so_counts

{'SO-001': 0, 'SO-002': 0}